In [22]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, udf, from_json
import requests

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
basePath='s3://mist-secorapp-production/'

paths = ["s3://mist-secorapp-production/ssr-application-summary-analytics/ssr-application-summary-analytics-production/dt=2022-03-13/", "s3://mist-secorapp-production/ssr-application-summary-analytics/ssr-application-summary-analytics-production/dt=2022-03-14/"]

df_application =spark.read.option("basePath",basePath).parquet(*paths)

df_application_teams = df_application.where(f.col("display_app_name") =='Microsoft Teams').select(f.col("mac"), f.col("display_app_name")).distinct()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
paths = ["s3://mist-secorapp-production/oc-stats-analytics/oc-stats-analytics-production/dt=2022-03-13/", "s3://mist-secorapp-production/oc-stats-analytics/oc-stats-analytics-production/dt=2022-03-14/"]


df_oc =spark.read.option("basePath",basePath).parquet(*paths)

df_oc_distinct = df_oc.select(f.col("mac"), f.col("network_interfaces"), f.col("ssr_peer_path_stats")).distinct()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_oc_intersect = df_oc_distinct.withColumn("explode_ssr_peer_path_stats", f.explode("ssr_peer_path_stats")).drop(f.col("ssr_peer_path_stats")).select(f.col("explode_ssr_peer_path_stats.network_interface"), f.col("mac"), f.col("network_interfaces"))

df_oc_intersect = df_oc_intersect.withColumn("explode_network_interfaces", f.explode("network_interfaces")).drop(f.col("network_interfaces")).withColumn("peer_ssr_network_interface", f.col("network_interface")).select(f.col("mac"), f.col("peer_ssr_network_interface"), f.col("explode_network_interfaces.interface_name"), f.col("explode_network_interfaces.name"), f.col("explode_network_interfaces.ips"), f.col("explode_network_interfaces.routing_instance")).distinct()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df_oc_intersect = df_oc_intersect.withColumn("oc_mac", f.col("mac")).drop(f.col("mac"))

df_application = df_application.select(f.col("mac"), f.col("display_app_name"))

df_join = df_oc_intersect.join(df_application, df_application["mac"] == df_oc_intersect["oc_mac"])

df_join = df_oc_intersect.join(df_application_teams, df_application_teams["mac"] == df_oc_intersect["oc_mac"]).distinct()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
schema = StructType([ 
    StructField("ip",StringType(),True), 
    StructField("version",StringType(),True), 
    StructField("city",StringType(),True), 
    StructField("country_code_iso3",StringType(),True), 
    StructField("latitude",StringType(),True), 
    StructField("longitude",StringType(),True), 
    StructField("timezone",StringType(),True), 
    StructField("country", StringType(), True),
    StructField("asn",StringType(),True), 
    StructField("org", StringType(), True),
    StructField("error", BooleanType(), True),
    StructField("reserved", BooleanType(), True)
  ])

@udf(returnType = StringType())
def get_response(ip):
	import json
	if ip:
		r  = requests.request('post', f'https://ipapi.co/{ip[-1]}/json/')
		if r.status_code == 200:
			return json.dumps(r.json())
		return json.dumps({"ip": ip, "error": r.status_code})
	return json.dumps({"ip": ip, "error": "no ip"})

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_result = df_join.withColumn("response", get_response(f.col("ips"))).withColumn("value", from_json(f.col("response"), schema)).select(f.col("peer_ssr_network_interface"), f.col("interface_name"),f.col("interface_name"), f.col("ips"),f.col("mac"), f.col("display_app_name"),f.col("value.*"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_result.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------+--------------+--------------+---------------+------------+----------------+-------------+-------+------+-----------------+----------+----------+----------------+-------+-------+-------------+-----+--------+
|peer_ssr_network_interface|interface_name|interface_name|ips            |mac         |display_app_name|ip           |version|city  |country_code_iso3|latitude  |longitude |timezone        |country|asn    |org          |error|reserved|
+--------------------------+--------------+--------------+---------------+------------+----------------+-------------+-------+------+-----------------+----------+----------+----------------+-------+-------+-------------+-----+--------+
|mpls                      |eth4-lan2     |eth4-lan2     |[192.168.100.1]|02000194b24d|Microsoft Teams |192.168.100.1|IPv4   |null  |null             |null      |null      |null            |null   |null   |null         |true |true    |
|broadband                 |eth2-wan1     |eth2-wan1    